# Fine-Tuning YOLOv11 With Our Dataset

#### Course: Deep Neural Engineering (IM1102)
#### Group: Ellen Cordemans, Ilse Harmers & Sem Pepels

The code in this notebook is adapted from [1].


---



**References**

[1] Kondrackis, L. (2024, December 5). How to Train YOLOv11 Instance
Segmentation on a Custom Dataset. Roboflow Blog. Retrieved April 15, 2025, from https://blog.roboflow.com/train-yolov11-instance-segmentation/

## Preparations

In [ ]:
# Checking the availability of Google Colab's GPU.
!nvidia-smi

In [ ]:
# Installing Roboflow and Ultralytics libraries.
!pip -q install "roboflow==1.1.58" "ultralytics==8.3.95"

In [ ]:
# Importing important models, functions and libraries.
from google.colab import userdata
from roboflow import Roboflow
import torch
from ultralytics import YOLO
import os
import re
from IPython.display import display
from PIL import Image
import locale

## Fine-Tune a YOLOv11 Segmentation Model

In [ ]:
# Downloading our dataset from Roboflow.
ROBOFLOW_API_KEY = userdata.get('ROBOFLOW_API_KEY')
rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace("workspace-v0o55").project("parsing-house-facades")
version = project.version(10)   # Train-Val-Test: 70-15-15.
dataset = version.download("yolov11")
dataset_path = f"{dataset.location}/data.yaml"

In [ ]:
# Checking whether cuda is available for Torch to use as device.
if torch.cuda.is_available():
    print("Cuda is available. Torch will use Cuda.")
    device = "cuda"
elif torch.backends.mps.is_available():
    print("MPS is available. Torch will use MPS.")
    device = "mps"
else:
    print("GPU is not available. Torch will fall back to CPU.")
    device = "cpu"

### Hyperparameter Tuning

In [ ]:
model = YOLO("Pretrained_models/yolo11l-seg.pt")

In [ ]:
# We wanted to perform a hyperparameter search, but the free resources provided by Google Colab
# are not enough to run this process within a reasonable time (i.e., < ~ 4 hours). Even when reducing
# the amount of parameters in the search, the runtime remains too lengthy for Colab's (free) GPU to handle.
search_space = {
    "lr0": (1e-5, 1e-1),
    "momentum": (0.6, 0.98),
    "weight_decay": (0.0, 0.001),
    "box": (0.02, 0.2),
    "cls": (0.2, 4.0),
}

model.tune(
    data=dataset_path,
    epochs=30,
    iterations=100,
    space=search_space,
    plots=False,
    save=False,
    val=False,
    device=device
)

### Fine-Tuning a YOLOv11 Segmentation Model Based On Our Dataset

In [ ]:
# For fine-tuning YOLOv11, we have chosen the second-to-last largest version as explained in the report.
# Note that running this cell takes a long while (~ 30 minutes).
results = model.train(data=dataset_path, epochs=100, imgsz=640, degrees=0.0, flipud=0.0, device=device)

In [ ]:
# Saving the train results to a ZIP file.
# This line might become deprecated when training multiple runs in one session, since the new directory will be saved as 'train{+1}'.
!zip -r /content/runs/segment/train/train.zip /content/runs/segment/train

### Testing Fine-Tuned Model

In [ ]:
# Determining metrics on the test set.
metrics = model.val(data=dataset_path, split="test")

In [ ]:
# Saving the test results to a ZIP file.
# This line might become deprecated when training multiple runs in one session, since the new directory will be saved as 'train{+1}'.
!zip -r /content/runs/segment/train2/test.zip /content/runs/segment/train2

In [ ]:
# In this cell, we display the quantitative results on the test set.
# Printing class labels.
print(f"Labels: {metrics.names}")

# Box AP metrics.
AP50b = metrics.box.ap50
AP50_95b = metrics.box.ap
# Segmentation AP metrics.
AP50s = metrics.seg.ap50
AP50_95s = metrics.seg.ap

print(f"AP@50: {AP50b} (B) & {AP50s} (S)")
print(f"AP@50-95: {AP50_95b} (B) & {AP50_95s} (S)\n")

# Box mAP metrics.
mAP50b = metrics.box.map50
mAP50_95b = metrics.box.map
# Segmentation mAP metrics.
mAP50s = metrics.seg.map50
mAP50_95s = metrics.seg.map

print(f"mAP@50: {mAP50b} (B) & {mAP50s} (S)")
print(f"mAP@50-95: {mAP50_95b} (B) & {mAP50_95s} (S)")

In [ ]:
# Displaying the confusion matrix of the test set for IoU = 0.45 (default value in Ultralytics library).
folder_segment_content = os.listdir("/content/runs/segment")
test_folder = None

# Extracting the most recently saved 'train' directory in which the test results were stored.
def extract_last_number(s):
    match = re.search(r'train(\d+)', s)
    if match:
        return int(match.group(1))
    elif s == "train":
        return -float('inf')
    return float('inf')

if len(folder_segment_content) == 1:
  test_folder == folder_segment_content[0]
else:
  filtered_list = [s for s in folder_segment_content if re.search(r'train\d*', s)]
  sorted_list = sorted(filtered_list, key=extract_last_number, reverse=True)
  test_folder = sorted_list[0]

# Displaying the (normalized) confusion matrix of the test data.
img = Image.open(f"/content/runs/segment/{test_folder}/confusion_matrix_normalized.png")
img_resized = img.resize((img.width // 4, img.height // 4))
display(img_resized)